接下来使用之前准备好的数据来训练一个yolo5模型，试试效果。首先把数据文件划分训练集、验证集、移到对应目录下。

对了，之前在目录里面创建了虚拟环境，跑之前记得选择笔记本的python环境。

In [1]:
# %cd ..
%pwd

'c:\\Users\\24760\\Desktop\\Researches\\DRL\\Zzz_PerfectAssist'

In [2]:
import os
import shutil
import random
import pandas as pd
from config import CSV_FILE, IMAGE_DIR, LABEL_DIR, TRAIN_IMAGE_DIR, VAL_IMAGE_DIR, TRAIN_LABEL_DIR, VAL_LABEL_DIR
from config import YOLOV5_DATA_CONFIG, YOLOV5_WEIGHTS, BATCH_SIZE, NUM_EPOCHS, LEARNING_RATE

In [ ]:
# 创建目标目录
os.makedirs(TRAIN_IMAGE_DIR, exist_ok=True)
os.makedirs(VAL_IMAGE_DIR, exist_ok=True)
os.makedirs(TRAIN_LABEL_DIR, exist_ok=True)
os.makedirs(VAL_LABEL_DIR, exist_ok=True)

# 读取CSV文件
df = pd.read_csv(CSV_FILE)

# 只保留 gold_flash 和 red_flash 类别
df = df[df['label'].isin(['gold_flash', 'red_flash'])]

# 获取不同类别的图像文件
class_0_images = df[df['label'] == 'gold_flash']['image'].tolist()
class_1_images = df[df['label'] == 'red_flash']['image'].tolist()

# 按 80/20 比例分为训练集和验证集
def split_dataset(images):
    random.shuffle(images)
    split_index = int(len(images) * 0.8)
    return images[:split_index], images[split_index:]

train_class_0, val_class_0 = split_dataset(class_0_images)
train_class_1, val_class_1 = split_dataset(class_1_images)

# 合并训练集和验证集
train_images = train_class_0 + train_class_1
val_images = val_class_0 + val_class_1

# 移动文件
def move_files(image_list, source_image_dir, source_label_dir, target_image_dir, target_label_dir):
    for image in image_list:
        image_path = os.path.join(source_image_dir, image)
        label_path = os.path.join(source_label_dir, os.path.splitext(image)[0] + '.txt')
        
        if os.path.exists(label_path):
            shutil.copy(image_path, target_image_dir)
            shutil.copy(label_path, target_label_dir)

move_files(train_images, IMAGE_DIR, LABEL_DIR, TRAIN_IMAGE_DIR, TRAIN_LABEL_DIR)
move_files(val_images, IMAGE_DIR, LABEL_DIR, VAL_IMAGE_DIR, VAL_LABEL_DIR)

print("文件移动完成。")

In [3]:
# %cp -r data/dataset yolo5/data
%cd yolov5

c:\Users\24760\Desktop\Researches\DRL\Zzz_PerfectAssist\yolov5


c:\Users\24760\Desktop\Researches\DRL\Zzz_PerfectAssist\myenv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

# 定义标签目录
label_dir = r'data\\dataset\\labels'

# 映射关系
label_mapping = {1: 0, 2: 1}

# 遍历标签目录中的所有文件
for subdir, _, files in os.walk(label_dir):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(subdir, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
            
            new_lines = []
            for line in lines:
                parts = line.split()
                class_id = int(parts[0])
                if class_id in label_mapping:
                    new_class_id = label_mapping[class_id]
                    parts[0] = str(new_class_id)
                    new_lines.append(' '.join(parts) + '\n')
                else:
                    new_lines.append(line)  # 保留其他类的标注
            
            # 写入更新后的标签文件
            with open(file_path, 'w') as f:
                f.writelines(new_lines)

print("标签更新完成。")


In [16]:
import os

# 定义标签目录
label_dir = r'data\\dataset\\labels'

# 标签统计
train_counts = {}
val_counts = {}

# 检查是否有标签2的存在
has_class_2 = False

# 遍历标签目录中的所有文件
for subdir, _, files in os.walk(label_dir):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(subdir, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
            
            for line in lines:
                parts = line.split()
                class_id = int(parts[0])
                
                # 检查是否有标签2的存在
                if class_id == 2:
                    has_class_2 = True
                    print(f"标签2在文件 {file_path} 中找到")

                # 更新标签统计
                if 'train' in subdir:
                    if class_id not in train_counts:
                        train_counts[class_id] = 0
                    train_counts[class_id] += 1
                elif 'val' in subdir:
                    if class_id not in val_counts:
                        val_counts[class_id] = 0
                    val_counts[class_id] += 1

# 打印标签统计信息
print("训练集标签统计：", train_counts)
print("验证集标签统计：", val_counts)

if not has_class_2:
    print("没有找到标签2")


训练集标签统计： {1: 13, 0: 54}
验证集标签统计： {1: 4, 0: 14}
没有找到标签2


接下来可以训练了

In [4]:
import torch
from IPython.display import Image, display
# 检查 GPU 是否可用
print(f"CUDA available: {torch.cuda.is_available()}")

CUDA available: False


开训！

In [ ]:
!python train.py --img 416 --batch-size {BATCH_SIZE} --epochs {NUM_EPOCHS} --data {YOLOV5_DATA_CONFIG} --weights {YOLOV5_WEIGHTS} --project runs/train --name exp

看看日志

In [ ]:
# 在终端里面跑下面的命令。
!tensorboard --logdir runs/train